In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score,precision_score,roc_auc_score,accuracy_score,confusion_matrix,classification_report
np.random.seed(90)

In [2]:
path = 'H:/Greyatom_sessions/jupyter workspace/train.tsv'
df = pd.read_csv(path,delimiter='\t')

FileNotFoundError: File b'H:/Greyatom_sessions/jupyter workspace/train.tsv' does not exist

In [3]:
import nltk

In [4]:
df1 = df.iloc[:500,:]
df1.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2



# Lower the words

In [7]:
df1['Phrase'] = df1['Phrase'].apply(lambda x:x.lower())

C:\Users\Lenovo\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


# Word tokenization

In [8]:
from nltk import word_tokenize
df2 = df1.copy()
df2['Phrase']=df2['Phrase'].apply(lambda x:word_tokenize(x))

In [9]:
df2['Phrase'].head()

0    [a, series, of, escapades, demonstrating, the,...
1    [a, series, of, escapades, demonstrating, the,...
2                                          [a, series]
3                                                  [a]
4                                             [series]
Name: Phrase, dtype: object

# StopWords

In [10]:
from nltk.corpus import stopwords
stop_words=set(stopwords.words('english'))
print(stop_words)

set([u'all', u'just', u"don't", u'being', u'over', u'both', u'through', u'yourselves', u'its', u'before', u'o', u'don', u'hadn', u'herself', u'll', u'had', u'should', u'to', u'only', u'won', u'under', u'ours', u'has', u"should've", u"haven't", u'do', u'them', u'his', u'very', u"you've", u'they', u'not', u'during', u'now', u'him', u'nor', u"wasn't", u'd', u'did', u'didn', u'this', u'she', u'each', u'further', u"won't", u'where', u"mustn't", u"isn't", u'few', u'because', u"you'd", u'doing', u'some', u'hasn', u"hasn't", u'are', u'our', u'ourselves', u'out', u'what', u'for', u"needn't", u'below', u're', u'does', u"shouldn't", u'above', u'between', u'mustn', u't', u'be', u'we', u'who', u"mightn't", u"doesn't", u'were', u'here', u'shouldn', u'hers', u"aren't", u'by', u'on', u'about', u'couldn', u'of', u"wouldn't", u'against', u's', u'isn', u'or', u'own', u'into', u'yourself', u'down', u"hadn't", u'mightn', u"couldn't", u'wasn', u'your', u"you're", u'from', u'her', u'their', u'aren', u"it's",

In [11]:
df2['Phrase'] = [[w for w in text if not w in stop_words] for text in df2['Phrase']]

# Lemmatization

In [12]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [13]:
df2['Phrase'] = [[wordnet_lemmatizer.lemmatize(w,pos = "v") for w in text ] for text in df2['Phrase']]

In [14]:
df2['Phrase'].head()

0    [series, escapades, demonstrate, adage, good, ...
1    [series, escapades, demonstrate, adage, good, ...
2                                             [series]
3                                                   []
4                                             [series]
Name: Phrase, dtype: object

# filling Missing Value

In [41]:
#df3 = df2.copy()
#df3['Phrase'] = df3[df3['Phrase']== ''] = 'Some value'
df2['Phrase'][df2['Phrase']== ''] = ['Some value']

C:\Users\Lenovo\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
df2['Phrase'].head()

0    series, escapades, demonstrate, adage, good, g...
1    series, escapades, demonstrate, adage, good, g...
2                                               series
3                                           Some value
4                                               series
Name: Phrase, dtype: object

In [ ]:
# a = df2['Phrase'][3]
# for i in range(0,len(df2['Phrase'])):
#     if df2['Phrase'][i] == a:
#         df2['Phrase'][i] = ['some value']
#     else:
#         df2['Phrase'][i] = df2['Phrase'][i]

# Making string

In [43]:
df2.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,"series, escapades, demonstrate, adage, good, g...",1
1,2,1,"series, escapades, demonstrate, adage, good, g...",2
2,3,1,series,2
3,4,1,Some value,2
4,5,1,series,2


In [29]:
df2['Phrase'] = df2['Phrase'].apply(lambda x: ', '.join(x))

In [40]:
df2['Phrase'].head()

0    series, escapades, demonstrate, adage, good, g...
1    series, escapades, demonstrate, adage, good, g...
2                                               series
3                                                     
4                                               series
Name: Phrase, dtype: object

In [30]:
X =df2['Phrase']
y = df2['Sentiment']


In [31]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 9)

In [32]:
from sklearn.feature_extraction.text import CountVectorizer 
from nltk.tokenize import TreebankWordTokenizer


In [33]:
vectorizer= CountVectorizer(analyzer = "word",   
                             tokenizer=TreebankWordTokenizer().tokenize ,    
                             preprocessor = None, 
                             stop_words = None,
                              ngram_range = (1,2),
                             max_features = 3000)

In [34]:
vectorizer.fit(X_train)
X_train1 = vectorizer.transform(X_train)


In [ ]:
#pd.DataFrame(X_train2.toarray(), columns=vectorizer.get_feature_names()).head(5)

In [35]:
X_test1 = vectorizer.transform(X_test)

In [44]:
# print(type(X_train1))
# pd.DataFrame(X_test1.toarray(), columns=vectorizer.get_feature_names()).head(5)

In [37]:

# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(max_depth = 50,random_state=9)
# clf.fit(X_train1, y_train)


from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth = 30,random_state=9)
clf.fit(X_train1, y_train)




RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=9, verbose=0, warm_start=False)

In [38]:
y_pred = clf.predict(X_test1)
y_pred_train = clf.predict(X_train1)
y_pred

array([3, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 3, 2, 2, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 3, 4, 1, 1,
       4, 1, 2, 2, 3, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 4, 2,
       2, 3, 2, 2, 1, 2, 2, 3, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2,
       0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [39]:
print('Accuracy test: {}'.format(accuracy_score(y_pred=y_pred,y_true= y_test)))
print('Accuracy train: {}'.format(accuracy_score(y_pred=y_pred_train,y_true= y_train)))
#print('Auc test: {}'.format(roc_auc_score(y_pred,y_test)))
print(confusion_matrix(y_pred=y_pred,y_true= y_test))

Accuracy test: 0.77
Accuracy train: 0.84
[[ 1  1  0  0  0]
 [ 2  8  6  0  0]
 [ 0  1 63  3  2]
 [ 0  0  5  3  0]
 [ 0  0  1  2  2]]


In [45]:
#from sklearn.datasets import load_files

In [ ]:
#load_files()